In [337]:
import copy

In [338]:
def setup_domains():
    domains = []  # domain for each cell
    avails = []  # size of the domain for each cell
    for i in range(9):
        domains.append([])
        avails.append([])
        for j in range(9):
            domains[i].append([])
            avails[i].append(9)
            for k in range(9):
                domains[i][j].append(True)
    return domains, avails

In [339]:
def read_data(filename, domains, avails):
    with open(filename) as f:
        data = f.readlines()

    board = []

    for i, line in enumerate(data):
        line = line.strip()  # remove newline character
        line = line.split("|")[1:-1]
        row = []
        for j, cell in enumerate(line):
            if cell == " ":
                row.append(0)  # 0 represents an empty cell
            else:
                cell = int(cell)
                row.append(cell)
                domains, avails = edit_domains(domains, avails, i, j, cell)
        board.append(row)
    
    return board, domains, avails

In [340]:
def display_board(board):
    for row in board:
        print("|", end="")
        for cell in row:
            print(cell, end="|")
        print()

In [341]:
def edit_domains(domains, avails, i, j, k):
    domains = copy.deepcopy(domains)
    avails = copy.deepcopy(avails)
    
    domains[i][j] = [False] * 9
    domains[i][j][k-1] = True
    avails[i][j] = 1
    
    # same column
    for x in range(9):
        y = j
        if x != i:
            if domains[x][y][k-1]:
                avails[x][y] -= 1
            domains[x][y][k-1] = False
    
    # same row
    for y in range(9):
        x = i
        if y != j:
            if domains[x][y][k-1]:
                avails[x][y] -= 1
            domains[x][y][k-1] = False
            
    # same square
    r = i // 3
    c = j // 3
    for x in range(3*r, 3*(r+1)):
        for y in range(3*c, 3*(c+1)):
            if x != i and y != j:
                if domains[x][y][k-1]:
                    avails[x][y] -= 1
                domains[x][y][k-1] = False
    
    return domains, avails

In [342]:
def check_consistency(avails):
    for i in range(9):
        for j in range(9):
            if avails[i][j] == 0:
                # print(f"DEBUG: cell ({i},{j}) is doomed!\n")
                return False
    return True

In [343]:
def edit_cell(num, board, domains, avails):
    if num >= 81:
        return True, board, domains, avails
    
    i = num // 9
    j = num % 9
    assert i * 9 + j == num
    
    board = copy.deepcopy(board)
    domains = copy.deepcopy(domains)
    avails = copy.deepcopy(avails)
    
    for k, val in enumerate(domains[i][j]):
        if val:
            board[i][j] = k+1
            new_domains, new_avails = edit_domains(domains, avails, i, j, k+1)
            
            # print(f"DEBUG: putting {k+1} in cell ({i},{j})")
            # display_board(board)
            # print()
            
            if check_consistency(new_avails):
                res, new_board, new_domains, new_avails = edit_cell(num + 1, board, new_domains, new_avails)
                if res:
                    return True, new_board, new_domains, new_avails
    # print(f"DEBUG: no other options in cell ({i},{j}).. going back to previous cell\n")
    return False, board, domains, avails

In [348]:
def solve(board, domains, avails):
    res, board, domains, avails = edit_cell(0, board, domains, avails)
    print(f"this sudoku puzzle solvable: {res}")
    display_board(board)

In [356]:
domains, avails = setup_domains()
board, domains, avails = read_data("input.txt", domains, avails)
display_board(board)

|8|0|0|9|3|0|0|0|2|
|0|0|9|0|0|0|0|4|0|
|7|0|2|1|0|0|9|6|0|
|2|0|0|0|0|0|0|9|0|
|0|6|0|0|0|0|0|7|0|
|0|7|0|0|0|6|0|0|5|
|0|2|7|0|0|8|4|0|6|
|0|3|0|0|0|0|5|0|0|
|5|0|0|0|6|2|0|0|8|


In [353]:
assert domains[0][0] == [False, False, False, False, False, False, False, True, False]
assert domains[0][1] == [True, False, False, True, True, False, False, False, False]
assert avails[0][0] == 1
assert avails[0][1] == 3

In [354]:
from time import perf_counter

In [357]:
start = perf_counter()
solve(board, domains, avails)
end = perf_counter()
print(f"time elapsed (s): {end - start}")

this sudoku puzzle solvable: True
|8|4|6|9|3|7|1|5|2|
|3|1|9|6|2|5|8|4|7|
|7|5|2|1|8|4|9|6|3|
|2|8|5|7|1|3|6|9|4|
|4|6|3|8|5|9|2|7|1|
|9|7|1|2|4|6|3|8|5|
|1|2|7|5|9|8|4|3|6|
|6|3|8|4|7|1|5|2|9|
|5|9|4|3|6|2|7|1|8|
time elapsed (s): 1.3229231999994226
